# Fitting the SOHO/VIRGO Solar Irradiance Power Spectrum

##### Brett Morris

In [ ]:
%matplotlib inline
import os
import json
from functools import partial

from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.time import Time
import astropy.units as u
from astropy.stats import mad_std

from gadfly import PowerSpectrum, scale
from gadfly.sun import broomhall_p_mode_freqs

from gadfly.psd import (
    linear_space_to_jax_parameterization, 
    jax_parameterization_to_linear_space,
    linear_space_to_dicts, ppm, to_psd_units,
)

from scipy.stats import binned_statistic
from lightkurve import LightCurve

from jax import jit, lax
import jax.numpy as jnp
from jax.scipy.optimize import minimize

from celerite2.jax import GaussianProcess, terms

The VIRGO/PMO6 1-minute time series is accessible online at: 

    ftp://ftp.pmodwrc.ch/pub/data/irradiance/virgo/old/1-minute_Data/VIRGO_1min_0083-7404.fits
    
We first load the VIRGO observations:

In [ ]:
hdu = fits.open('data/VIRGO_1min_0083-7404.fits.gz')
raw_fluxes = hdu[0].data
header = hdu[0].header

header

We can reconstruct the times from the background info in the header. We will linearly interpolate over missing measurements. 

In [ ]:
soho_mission_day = Time("1995-12-1 00:00")

times = (
    soho_mission_day.jd + 
    header['TIME'] + 
    np.arange(header['NAXIS1']) / 1440
)
times_astropy = Time(times, format='jd')

fluxes = raw_fluxes.copy()
interp_fluxes = np.interp(
    times[raw_fluxes == -99], times[raw_fluxes != -99], fluxes[raw_fluxes != -99]
)
d = (times[1] - times[0]) * u.day

fluxes[raw_fluxes == -99] = interp_fluxes

fluxes = 1e6 * (fluxes / np.median(fluxes) - 1) * ppm
fluxes_std_ppm = mad_std(fluxes.value)

skip_every = 500
plt.plot_date(
    times_astropy.plot_date[::skip_every], 
    fluxes[::skip_every], fmt='.'
)
plt.xlabel('Date')
plt.ylabel(f'Flux [{fluxes.unit}]');

Compute the full and binned power spectrum: 

In [ ]:
solar_light_curve = LightCurve(
    time=times_astropy, 
    flux=fluxes
)

solar_power_spectrum = PowerSpectrum.from_light_curve(
    solar_light_curve, detrend=False
)
# this binned power spectrum will be used for fitting low-frequency
# components ([super-/meso-/]granulation)
solar_power_spectrum_binned = solar_power_spectrum.bin(len(fluxes) // 10000, constant=1)

In [ ]:
n_p_mode_bins = 5000

# This binned, cropped power spectrum is useful for fitting p-modes
p_mode_binned = solar_power_spectrum.cutout(2000*u.uHz, 4000*u.uHz).bin(n_p_mode_bins)

# This binned, cropped power spectrum is useful for plotting the p-mode fit
p_mode_bin_for_plot = solar_power_spectrum.cutout(1800*u.uHz, 4000*u.uHz).bin(n_p_mode_bins)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
solar_power_spectrum_binned.plot(
    ax=ax, p_mode_inset=False
)

Narrow our set some boundaries in frequency over which we will fit the power spectrum:

In [ ]:
# Set the max frequency to be near the peak in the p-modes
cutoff_freq_max = 3000 * u.uHz
# Set the min frequency to be low
cutoff_freq_min = 0.1 * u.uHz

in_bounds = (
    (solar_power_spectrum_binned.frequency < cutoff_freq_max) &
    (solar_power_spectrum_binned.frequency > cutoff_freq_min)
)

y = to_psd_units(
    solar_power_spectrum_binned.power[in_bounds]
).value

yerr = to_psd_units(
    solar_power_spectrum_binned.error[in_bounds]
).value

x = solar_power_spectrum_binned.frequency[in_bounds].to(u.uHz).value

# # Make sure no NaNs make it into the calculations below:
mask_y = np.logical_not(np.isnan(y))
x = x[mask_y]

yerr_obs = yerr[mask_y].copy()

# The uncertainties produced during the binning in `yerr`
# are good for high frequencies, where the statistics are 
# reliable. We can estimate a "noise floor" as a function
# of frequency, which we've done by eye and has these parameters:
alpha = 8e3
beta = -1.7
yerr = np.nanmax([yerr[mask_y], alpha * x[mask_y] ** beta], axis=0)

# Now take the min of the the yerrs and the noise floor:
y = y[mask_y]
noise_floor = 0.1
mask_fix_yerr = np.isnan(yerr) | (yerr <= 0) | (yerr / y < noise_floor)
yerr[mask_fix_yerr] = noise_floor * y[mask_fix_yerr]

noise_floor_plot = False

if noise_floor_plot:
    plt.loglog(x, yerr)
    plt.loglog(x, yerr_obs)
    plt.plot(x, alpha * x ** beta)

Fit the low-frequency features in the solar power spectrum.

In [ ]:
# Get the frequencies at the p-mode peaks from Broomhall et al. (2009):
p_mode_freqs = broomhall_p_mode_freqs()

# We'll scale the uncertainties near the p-modes by a factor
# proportional to the amplitudes of the p-mode envelope:
errorbar_scale_factor = (
    50 * scale.p_mode_intensity_kjeldsen_bedding(x * u.uHz).value/20.1 + 1
)

In [ ]:
# This fixed value for Q should be close to the critical value of 0.5:
fixed_Q = 0.6

min_p_mode = 2000  # Assume p-modes start about here in uHz
max_p_mode = 4000  # Assume p-modes end about here in uHz

mask_p_modes = (x < min_p_mode) | (x > max_p_mode)

def sho(S0, w0, Q=fixed_Q):
    """
    Underdamped SHO kernel from celerite2
    """
    return terms.UnderdampedSHOTerm(
        S0=S0, w0=w0, Q=Q
    )

def low_freq_kernels_five_kernels(p):
    """
    Sum of five underdamped SHO kernels, meant for fitting
    low frequency features in the solar power spectrum.
    """
    delta_S0_0, w0_0 = p[0:2]
    delta_S0_1, delta_w0_1 = p[2:4]
    delta_S0_2, delta_w0_2 = p[4:6]
    delta_S0_3, delta_w0_3 = p[6:8]
    S0_4, delta_w0_4 = p[8:10]
    
    S0_3 = 10 ** delta_S0_3 + S0_4
    S0_2 = 10 ** delta_S0_2 + S0_3
    S0_1 = 10 ** delta_S0_1 + S0_2
    S0_0 = 10 ** delta_S0_0 + S0_1
    
    w0_1 = w0_0 + 10 ** delta_w0_1
    w0_2 = w0_1 + 10 ** delta_w0_2
    w0_3 = w0_2 + 10 ** delta_w0_3
    w0_4 = w0_3 + 10 ** delta_w0_4
    
    kernel = terms.TermSum(
        sho(S0_0, w0_0),
        sho(S0_1, w0_1), 
        sho(S0_2, w0_2),
        sho(S0_3, w0_3),
        sho(S0_4, w0_4),
    )
    
    return kernel

def jax_model_low_freq(p, omega):
    """
    Model of the low-frequency power spectrum, computed
    at angular frequencies ``omega``.
    """
    psd = low_freq_kernels_five_kernels(p).get_psd(omega)
    return psd

def jax_model_low_freq_terms(p, omega):
    """
    Model of the low-frequency power spectrum, computed
    at angular frequencies ``omega``.
    """
    psds = [trm.get_psd(omega) for trm in low_freq_kernels_five_kernels(p).terms]
    return psds

@partial(jit, static_argnums=(1, 2, 3))
def chi2_low_freq_model(
    p, 
    y=y, omega=2*np.pi*x, yerr=jnp.where(mask_p_modes, 1, errorbar_scale_factor) * yerr
):
    """
    chi^2 of the low-frequency model
    """
    chi2_result = jnp.nansum( 
        (y - jax_model_low_freq(p, omega=omega))**2  / yerr**2
    )
    return chi2_result


fit = True

all_S0s = 1.5 * 10 ** np.array([4.1, 1.2, -0.3, -0.8, -1.5])
all_omegas = np.array([5e0, 9.5e1, 6e2, 6.3e3, 2.0e4])

initp = linear_space_to_jax_parameterization(all_S0s, all_omegas)

fig, ax = plt.subplots(figsize=(12, 5))

if fit:
    result = minimize(
        chi2_low_freq_model, jnp.array(initp), method='bfgs',
    )
    print('Fit successful:', result.success)
    if result.status == 3:
        # status message keys source/docs
        # https://github.com/scipy/scipy/blob/85d25b6e4a9b95371e48bae75c19459a0b77d18e/scipy/optimize/_optimize.py#L1239-L1242
        print('Warning: nans encountered in optimization')
    bestp_lowfreq = result.x

def numpy_model(p, omega=2*np.pi*x):
    return np.array(jax_model_low_freq(p, omega=omega))

def numpy_model_terms(p, omega=2*np.pi*x):
    return jax_model_low_freq_terms(p, omega)

if fit:
    ax.loglog(solar_power_spectrum_binned.frequency.value, 
              numpy_model(bestp_lowfreq, omega=2*np.pi*solar_power_spectrum_binned.frequency.value), 
              color='r', lw=6, label='Fit', zorder=0)
    for trm in numpy_model_terms(bestp_lowfreq, omega=2*np.pi*solar_power_spectrum_binned.frequency.value):
        ax.loglog(solar_power_spectrum_binned.frequency.value, np.array(trm), color='cyan', lw=1, zorder=0)
ax.loglog(x, numpy_model(initp), color='C0', ls=':', label='Init', zorder=10)
ax.errorbar(solar_power_spectrum_binned.frequency.value, solar_power_spectrum_binned.power.value, None, 
            color='k', ecolor='silver', label='Binned', fmt='.', ms=4, mfc='none', rasterized=True, zorder=10)
ax.axvspan(min_p_mode, max_p_mode, alpha=0.2)
ax.set_xlabel('Frequency ($\mu$Hz)')
ax.set_ylabel('Power')
ax.set_xlim([cutoff_freq_min.value, 5000])
ax.set_ylim([1e-3, 1e5])
plt.legend()

Helpful diagnostic plot for understanding if the errorbars are over/underestimated:

In [ ]:
yerr_diagnostic_plot = True

if yerr_diagnostic_plot: 
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.loglog(
        x, np.abs((y - numpy_model(bestp_lowfreq)) / yerr), 'k.',
    )
    ax.set(
        xlabel='Freq ($\\mu$Hz)', ylabel='abs((O - C) / E)'
    )
    ax.axvspan(min_p_mode, max_p_mode, alpha=0.2)

Define functions for the p-mode only fit:

In [ ]:
nu_max = 3079.76 * u.uHz  # central peak p-mode frequency from Kiefer 2018

# Scale factor for the the p-mode peak amplitudes:
scaled_intensities = scale.p_mode_intensity_kjeldsen_bedding(p_mode_freqs).value/20.1

log_Q_left = 3.1
log_Q_right = 3.3
log_S0_left_0 = -5.5
log_S0_left_1 = -6.4
log_S0_right = -5.4

@jit
def sho_psd(omega, S0, w0, Q):
    """
    Stochastically driven, dampled harmonic oscillator.
    """
    # What follows is the usual celerite2 SHO PSD:
    return np.sqrt(2/np.pi) * S0 * w0**4 / ((omega**2 - w0**2)**2 + (omega**2 * w0**2 / Q**2))

@jit
def insert_omegas(initp, omegas):
    log_S0_left_0, log_S0_left_1, log_S0_right, log_Q_left, log_Q_right = initp
    n = len(omegas)
    n_left = n // 2
    n_left_0 = n_left // 2
    n_left_1 = n_left - n_left_0
    n_right = n - n_left
    
    # The naming of the variables needs to be improved
    # for clarity:
    log_S0_left_concat = jnp.concatenate([
        jnp.broadcast_to(log_S0_left_0, (n_left_0, )), 
        jnp.broadcast_to(log_S0_left_1, (n_left_1, ))
    ])
    log_S0s = jnp.concatenate([
        log_S0_left_concat, jnp.broadcast_to(log_S0_right, (n_right, ))
    ])
    left_omegas = omegas[1:][::2]
    left_0_omegas = left_omegas[1:][::2]
    left_1_omegas = left_omegas[0:][::2]
    right_omegas = omegas[0:][::2]
    reordered_omegas = jnp.concatenate([
        left_0_omegas, left_1_omegas, right_omegas
    ])
    log_Qs = jnp.concatenate([
        jnp.broadcast_to(log_Q_left, (n_left,)), 
        jnp.broadcast_to(log_Q_right, (n_right, ))
    ])
    out = jnp.vstack([log_S0s, reordered_omegas, log_Qs]).T
    return out[jnp.argsort(reordered_omegas)]

@jit 
def scaled(hyperparams, scaled_intensities=scaled_intensities):
    """
    Scale the S0 hyperparameters by their scaling in
    Kiefer et al. (2018)
    """
    scaled_S0_column = hyperparams[:, 0] + jnp.log10(scaled_intensities)
    return jnp.vstack([scaled_S0_column, hyperparams[:, 1:].T]).T

@jit
def p_mode_model(p, omega, p_mode_frequencies=p_mode_freqs.value):
    sho_hyperparams = insert_omegas(p, omegas=2*np.pi*p_mode_frequencies)
    sho_hyperparams = scaled(sho_hyperparams)
    def f(carry, x):
        log_S0, w0, log_Q = x
        return carry, sho_psd(omega, jnp.power(10.0, log_S0), w0, jnp.power(10.0, log_Q))
    return jnp.sum(lax.scan(f, 0, sho_hyperparams)[1], axis=0) + 1

In [ ]:
initp = jnp.array([log_S0_left_0, log_S0_left_1, log_S0_right, log_Q_left, log_Q_right])

To do the p-mode only fit, we will normalize out the low-frequency fit. Also normalize out a polynomial baseline trend.

In [ ]:
norm_lowfreq = low_freq_kernels_five_kernels(bestp_lowfreq).get_psd(2*np.pi*p_mode_binned.frequency) 

poly_order = 5
poly_fit_x = (p_mode_binned.frequency - p_mode_binned.frequency.mean()).value
poly_fit_y = p_mode_binned.power / norm_lowfreq
norm_corr = np.polyval(np.polyfit(
    poly_fit_x,
    poly_fit_y,
    poly_order,
    w=1/poly_fit_y/p_mode_binned.error,
), poly_fit_x)

p_mode_binned_normed = PowerSpectrum(p_mode_binned.frequency, poly_fit_y / norm_corr, error=3 * mad_std(poly_fit_y, ignore_nan=True))

Using the model generated from the initial parameters, we'll mask out power spectrum measurements that are close to the background by masking out observations where the excess power is expected to be small.

In [ ]:
init_model = p_mode_model(initp, omega=2*np.pi*p_mode_binned.frequency.value)

mask_below_background_threshold = False

if mask_below_background_threshold:
    background_threshold = 0.01
    near_modes = (
        init_model > 1 + 
        background_threshold * 
        mad_std(p_mode_binned_normed.power.value, ignore_nan=True)
    )

    plt.figure(figsize=(20, 6))
    plt.errorbar(
        p_mode_binned_normed.frequency[near_modes], 
        p_mode_binned_normed.power[near_modes], 
        p_mode_binned_normed.error, 
        fmt='.', ecolor='silver')

    plt.plot(p_mode_binned.frequency, init_model, zorder=5)
    plt.ylim([0, 15])
else: 
    near_modes = np.ones_like(init_model).astype(bool)

In [ ]:
def chi2_p_modes(
    p,
    y=p_mode_binned_normed.power.value[near_modes], 
    omega=2*np.pi*p_mode_binned_normed.frequency.value[near_modes], 
    yerr=p_mode_binned_normed.error.value
):
    """
    chi^2 of the low-frequency model
    """
    chi2_result = jnp.nansum( 
        (y - p_mode_model(p, omega=omega))**2 / yerr**2
    )
    return chi2_result

Run the fit for the p-mode oscillations:

In [ ]:
result = minimize(chi2_p_modes, initp, method='bfgs') 
if result.success:
    print("Fit successful!")

# These are the best-fit parameters:
bestp_p_modes = result.x
bestp_p_modes

In [ ]:
def p_mode_dicts(p, p_mode_frequencies=p_mode_freqs.value):
    sho_hyperparams = insert_omegas(p, omegas=2*np.pi*p_mode_frequencies)
    sho_hyperparams = scaled(sho_hyperparams)
    hp = []
    for x in sho_hyperparams:
        log_S0, w0, log_Q = x
        background = np.float64(low_freq_kernels_five_kernels(bestp_lowfreq).get_psd(w0))
        hp.append(dict(S0=np.power(10.0, log_S0) * background, w0=np.float64(w0), Q=np.power(10.0, log_Q)))
    return hp

p_mode_list = p_mode_dicts(result.x)

In [ ]:
# Convert the best-fit parameters from the low-frequencies to an array
# with shape (5, 3) = (n_terms, n_hyperparams_per_term)
bestp_lowfreq_array = np.vstack([jax_parameterization_to_linear_space(bestp_lowfreq), fixed_Q * np.ones(5)]).T

# convert the array format hyperparams to a list of dict:
bestp_lowfreq_list = []

for S0, w0, Q in bestp_lowfreq_array:
    bestp_lowfreq_list.append(
        dict(
            hyperparameters=dict(
                S0=S0, w0=w0, Q=Q
            ), 
            metadata=dict(
                fixed_parameters=['Q'] if Q == fixed_Q else ['w0']
            )
        )
    )

In [ ]:
all_hyperparameters = [*bestp_lowfreq_list]

for p in p_mode_list:
    all_hyperparameters.append(
        dict(
            hyperparameters=p, 
            metadata=dict(
                fixed_parameters=['Q'] if p['Q'] == fixed_Q else ['w0']
            )
        )
    )

In [ ]:
all_terms = []

for trm in low_freq_kernels_five_kernels(bestp_lowfreq).terms:
    all_terms.append(trm)

for hp in p_mode_list:
    all_terms.append(terms.UnderdampedSHOTerm(**hp))

In [ ]:
kernel_all_terms = terms.TermSum(*all_terms)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
solar_power_spectrum_binned.plot(ax=ax[0], freq=solar_power_spectrum_binned.frequency, p_mode_inset=False)
ax[0].plot(
    solar_power_spectrum_binned.frequency.value, 
    kernel_all_terms.get_psd(2 * np.pi * solar_power_spectrum_binned.frequency.value)
)

p_mode_bin_for_plot.plot(
    ax=ax[1], p_mode_inset=False, scaling_low_freq='semilogy', 
)
ax[1].plot(
    p_mode_bin_for_plot.frequency.value, 
    kernel_all_terms.get_psd(2 * np.pi * p_mode_bin_for_plot.frequency.value)
)
ax[1].set_xlim(1800, 4000)
ax[1].set_ylim(1e-2, 1)

In [ ]:
overwrite = False
parameter_vector_path = 'hyperparameters.json'
if not os.path.exists(parameter_vector_path) or overwrite:
    with open('hyperparameters.json', 'w') as w:
        json.dump(all_hyperparameters, w, indent=4)

else: 
    print('skipping overwrite')